In [83]:
"""
Author      : Yi-Chieh Wu, Sriram Sankararman
Description : Twitter
"""

from string import punctuation

import numpy as np
import matplotlib.pyplot as plt
# !!! MAKE SURE TO USE LinearSVC.decision_function(X), NOT LinearSVC.predict(X) !!!
# (this makes ''continuous-valued'' predictions)
from sklearn.svm import LinearSVC
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics

In [90]:
import warnings

#I used this to supress warnings from LinearSVC regarding some future updates in the library.
warnings.filterwarnings("ignore", category=FutureWarning, module='sklearn.svm')

# Problem 3: Twitter Analysis Using SVM

In [91]:
######################################################################
# functions -- input/output
######################################################################

def read_vector_file(fname):
    """
    Reads and returns a vector from a file.

    Parameters
    --------------------
        fname  -- string, filename

    Returns
    --------------------
        labels -- numpy array of shape (n,)
                    n is the number of non-blank lines in the text file
    """
    return np.genfromtxt(fname)


def write_label_answer(vec, outfile):
    """
    Writes your label vector to the given file.

    Parameters
    --------------------
        vec     -- numpy array of shape (n,) or (n,1), predicted scores
        outfile -- string, output filename
    """

    # for this project, you should predict 70 labels
    if(vec.shape[0] != 70):
        print("Error - output vector should have 70 rows.")
        print("Aborting write.")
        return

    np.savetxt(outfile, vec)
    

In [92]:
######################################################################
# functions -- feature extraction
######################################################################

def extract_words(input_string):
    """
    Processes the input_string, separating it into "words" based on the presence
    of spaces, and separating punctuation marks into their own words.

    Parameters
    --------------------
        input_string -- string of characters

    Returns
    --------------------
        words        -- list of lowercase "words"
    """

    for c in punctuation :
        input_string = input_string.replace(c, ' ' + c + ' ')
    return input_string.lower().split()


def extract_dictionary(infile):
    """
    Given a filename, reads the text file and builds a dictionary of unique
    words/punctuations.

    Parameters
    --------------------
        infile    -- string, filename

    Returns
    --------------------
        word_list -- dictionary, (key, value) pairs are (word, index)
    """

    word_list = {}
    idx = 0
    with open(infile, 'r') as fid :
        # process each line to populate word_list
        for input_string in fid:
            words = extract_words(input_string)
            for word in words:
                if word not in word_list:
                    word_list[word] = idx
                    idx += 1
    return word_list


def extract_feature_vectors(infile, word_list):
    """
    Produces a bag-of-words representation of a text file specified by the
    filename infile based on the dictionary word_list.

    Parameters
    --------------------
        infile         -- string, filename
        word_list      -- dictionary, (key, value) pairs are (word, index)

    Returns
    --------------------
        feature_matrix -- numpy array of shape (n,d)
                          boolean (0,1) array indicating word presence in a string
                            n is the number of non-blank lines in the text file
                            d is the number of unique words in the text file
    """

    num_lines = sum(1 for line in open(infile,'r'))
    num_words = len(word_list)
    feature_matrix = np.zeros((num_lines, num_words))

    with open(infile, 'r') as fid :
        # process each line to populate feature_matrix
        for i, input_string in enumerate(fid):
            words = extract_words(input_string)
            for word in words:
                feature_matrix[i, word_list[word]] = 1.0

    return feature_matrix

In [93]:
######################################################################
# functions -- evaluation
######################################################################

def performance(y_true, y_pred, metric="accuracy"):
    """
    Calculates the performance metric based on the agreement between the
    true labels and the predicted labels.

    Parameters
    --------------------
        y_true -- numpy array of shape (n,), known labels
        y_pred -- numpy array of shape (n,), (continuous-valued) predictions
        metric -- string, option used to select the performance measure
                  options: 'accuracy', 'f1-score', 'auroc', 'precision',
                           'sensitivity', 'specificity'

    Returns
    --------------------
        score  -- float, performance score
    """
    # map continuous-valued predictions to binary labels
    y_label = np.sign(y_pred)
    y_label[y_label==0] = 1

    ### ========== TODO : START ========== ###
    # part 1a: compute classifier performance
    if metric == "accuracy":
        return metrics.accuracy_score(y_true, y_label)
    elif metric == "f1-score":
        return metrics.f1_score(y_true, y_label)
    elif metric == "auroc":
        return metrics.roc_auc_score(y_true, y_pred)
    elif metric == "precision":
        return metrics.precision_score(y_true, y_label)
    elif metric == "sensitivity":
        return metrics.recall_score(y_true, y_label)
    elif metric == "specificity":
        confusion_matrix = metrics.confusion_matrix(y_true, y_label).ravel()
        tn, fp, fn, tp = confusion_matrix.ravel()
        return (tn)/(tn + fp)

    ### ========== TODO : END ========== ###


def cv_performance(clf, X, y, kf, metric="accuracy"):
    """
    Splits the data, X and y, into k-folds and runs k-fold cross-validation.
    Trains classifier on k-1 folds and tests on the remaining fold.
    Calculates the k-fold cross-validation performance metric for classifier
    by averaging the performance across folds.

    Parameters
    --------------------
        clf    -- classifier (instance of LinearSVC)
        X      -- numpy array of shape (n,d), feature vectors
                    n = number of examples
                    d = number of features
        y      -- numpy array of shape (n,), binary labels {1,-1}
        kf     -- model_selection.StratifiedKFold
        metric -- string, option used to select performance measure

    Returns
    --------------------
        score   -- float, average cross-validation performance across k folds
    """

    ### ========== TODO : START ========== ###
    # part 1b: compute average cross-validation performance
    accuracy_scores = []
    for train_indices, test_indices in kf.split(X,y):
        x_train = X[train_indices]
        x_test = X[test_indices]
        y_train = y[train_indices]
        y_test = y[test_indices]
        clf.fit(x_train, y_train)
        y_pred = clf.decision_function(x_test)
        accuracy_scores.append(performance(y_test, y_pred, metric))
    
    return np.mean(accuracy_scores)
    ### ========== TODO : END ========== ###


def select_param_linear(X, y, kf, metric="accuracy"):
    """
    Sweeps different settings for the hyperparameter of a linear SVM,
    calculating the k-fold CV performance for each setting, then selecting the
    hyperparameter that 'maximize' the average k-fold CV performance.

    Parameters
    --------------------
        X      -- numpy array of shape (n,d), feature vectors
                    n = number of examples
                    d = number of features
        y      -- numpy array of shape (n,), binary labels {1,-1}
        kf     -- model_selection.StratifiedKFold
        metric -- string, option used to select performance measure

    Returns
    --------------------
        C -- float, optimal parameter value for linear SVM
    """

    print('Linear SVM Hyperparameter Selection based on ' + str(metric) + ':')
    C_range = 10.0 ** np.arange(-3, 3)

    ### ========== TODO : START ========== ###
    # part 1c: select optimal hyperparameter using cross-validation
    best_C = None
    best_accuracy = -(np.inf)
    for c in C_range:
        clf = LinearSVC(loss='hinge', random_state = 0, C = c)
        accuracy_of_clf = cv_performance(clf, X, y, kf, metric)
        if accuracy_of_clf > best_accuracy:
            best_accuracy = accuracy_of_clf
            best_C = c
    
    return best_C
    ### ========== TODO : END ========== ###


def performance_test(clf, X, y, metric="accuracy"):
    """
    Estimates the performance of the classifier.

    Parameters
    --------------------
        clf          -- classifier (instance of LinearSVC)
                          [already fit to data]
        X            -- numpy array of shape (n,d), feature vectors of test set
                          n = number of examples
                          d = number of features
        y            -- numpy array of shape (n,), binary labels {1,-1} of test set
        metric       -- string, option used to select performance measure

    Returns
    --------------------
        score        -- float, classifier performance
    """


    ### ========== TODO : START ========== ###
    # part 2b: return performance on test data under a metric.
    y_pred = clf.decision_function(X)
    return performance(y, y_pred, metric)
    ### ========== TODO : END ========== ###

In [110]:
######################################################################
# main
######################################################################

def main() :
    np.random.seed(1234)

    # read the tweets and its labels, change the following two lines to your own path.
    ### ========== TODO : START ========== ###
    file_path = '../data/tweets.txt'
    label_path = '../data/labels.txt'
    ### ========== TODO : END ========== ###
    dictionary = extract_dictionary(file_path)
    print(len(dictionary))
    X = extract_feature_vectors(file_path, dictionary)
    y = read_vector_file(label_path)
    # split data into training (training + cross-validation) and testing set
    X_train, X_test = X[:560], X[560:]
    y_train, y_test = y[:560], y[560:]

    metric_list = ["accuracy", "f1-score", "auroc", "precision", "sensitivity", "specificity"]

    ### ========== TODO : START ========== ###
    # part 1b: create stratified folds (5-fold CV)
    print()
    kf = StratifiedKFold(n_splits=5)
    clf = LinearSVC(loss='hinge', random_state=0)
    best_C_values = {}
    for metric in metric_list:
        print(metric,": ",cv_performance(clf, X_train, y_train, kf, metric))
    print("The reason why we'd want to maintain class proportion across folds is because doing so ensures that the trends learnt arent biased towards the more prominent data set and also ensures that the model is better suited to generalize to test data which might not be as skewed")
    # part 1c: for each metric, select optimal hyperparameter for linear SVM using CV
    #using kf from part 1(b):
    print()
    for metric in metric_list:
        best_C = select_param_linear(X_train, y_train, kf, metric)
        best_C_values[metric] = best_C
        print(f"Best C for {metric} is {best_C}")
    # part 2a: train linear SVMs with selected hyperparameters
    best_classifiers_per_metric = {}
    for metric in metric_list:
        clf = LinearSVC(loss='hinge', random_state=0, C=best_C_values[metric])
        clf.fit(X_train, y_train)
        best_classifiers_per_metric[metric] = clf
    # part 2b: test the performance of your classifiers.
    print()
    for metric in metric_list:
        clf = best_classifiers_per_metric[metric]
        print(f"Performance of best classifier based on {metric} is ", performance_test(clf, X_test, y_test, metric))
    ### ========== TODO : END ========== ###


if __name__ == "__main__" :
    main()

1811

accuracy :  0.8196428571428571
f1-score :  0.8776260952681142
auroc :  0.8717871655638666
precision :  0.858434814720795
sensitivity :  0.9042088607594936
specificity :  0.612310606060606
The reason why we'd want to maintain class proportion across folds is because doing so ensures that the trends learnt arent biased towards the more prominent data set and also ensures that the model is better suited to generalize to test data which might not be as skewed

Linear SVM Hyperparameter Selection based on accuracy:
Best C for accuracy is 1.0
Linear SVM Hyperparameter Selection based on f1-score:
Best C for f1-score is 1.0
Linear SVM Hyperparameter Selection based on auroc:
Best C for auroc is 1.0
Linear SVM Hyperparameter Selection based on precision:
Best C for precision is 10.0
Linear SVM Hyperparameter Selection based on sensitivity:
Best C for sensitivity is 0.001
Linear SVM Hyperparameter Selection based on specificity:
Best C for specificity is 1.0

Performance of best classifie

# Problem 4: Boosting vs. Decision Tree

In [95]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.model_selection import cross_val_score, train_test_split

In [96]:
class Data :
    
    def __init__(self) :
        """
        Data class.
        
        Attributes
        --------------------
            X -- numpy array of shape (n,d), features
            y -- numpy array of shape (n,), targets
        """
                
        # n = number of examples, d = dimensionality
        self.X = None
        self.y = None
        
        self.Xnames = None
        self.yname = None
    
    def load(self, filename, header=0, predict_col=-1) :
        """Load csv file into X array of features and y array of labels."""
        
        # determine filename
        f = filename
        
        # load data
        with open(f, 'r') as fid :
            data = np.loadtxt(fid, delimiter=",", skiprows=header)
        
        # separate features and labels
        if predict_col is None :
            self.X = data[:,:]
            self.y = None
        else :
            if data.ndim > 1 :
                self.X = np.delete(data, predict_col, axis=1)
                self.y = data[:,predict_col]
            else :
                self.X = None
                self.y = data[:]
        
        # load feature and label names
        if header != 0:
            with open(f, 'r') as fid :
                header = fid.readline().rstrip().split(",")
                
            if predict_col is None :
                self.Xnames = header[:]
                self.yname = None
            else :
                if len(header) > 1 :
                    self.Xnames = np.delete(header, predict_col)
                    self.yname = header[predict_col]
                else :
                    self.Xnames = None
                    self.yname = header[0]
        else:
            self.Xnames = None
            self.yname = None


# helper functions
def load_data(filename, header=0, predict_col=-1) :
    """Load csv file into Data class."""
    data = Data()
    data.load(filename, header=header, predict_col=predict_col)
    return data

In [97]:
# Change the path to your own data directory
### ========== TODO : START ========== ###
titanic = load_data("../data/titanic_train.csv", header=1, predict_col=0)
### ========== TODO : END ========== ###
X = titanic.X; Xnames = titanic.Xnames
y = titanic.y; yname = titanic.yname
n,d = X.shape  # n = number of examples, d =  number of features

In [98]:
def error(clf, X, y, ntrials=100, test_size=0.2) :
    """
    Computes the classifier error over a random split of the data,
    averaged over ntrials runs.

    Parameters
    --------------------
        clf         -- classifier
        X           -- numpy array of shape (n,d), features values
        y           -- numpy array of shape (n,), target classes
        ntrials     -- integer, number of trials
        test_size   -- proportion of data used for evaluation

    Returns
    --------------------
        train_error -- float, training error
        test_error  -- float, test error
    """

    train_error = 0
    test_error = 0

    train_scores = []; test_scores = [];
    for i in range(ntrials):
        xtrain, xtest, ytrain, ytest = train_test_split (X,y, test_size = test_size, random_state = i)
        clf.fit (xtrain, ytrain)

        ypred = clf.predict (xtrain)
        err = 1 - metrics.accuracy_score (ytrain, ypred, normalize = True)
        train_scores.append (err)

        ypred = clf.predict (xtest)
        err = 1 - metrics.accuracy_score (ytest, ypred, normalize = True)
        test_scores.append (err)

    train_error =  np.mean (train_scores)
    test_error = np.mean (test_scores)
    return train_error, test_error


In [99]:
### ========== TODO : START ========== ###
# Part 4(a): Implement the decision tree classifier and report the training error.
print('Classifying using Decision Tree...')
decision_tree_clf = DecisionTreeClassifier(criterion="entropy", random_state=0)
decision_tree_clf.fit(X,y)
y_pred = decision_tree_clf.predict(X)
#Used accuracy_score instead of error because question asked us to calculate the training error
print("Training error is ",(1-metrics.accuracy_score(y, y_pred)))
_ , test_error = error(decision_tree_clf, X, y)
print("Testing error is", test_error)
### ========== TODO : END ========== ###

Classifying using Decision Tree...
Training error is  0.014044943820224698
Testing error is 0.24104895104895108


In [102]:
### ========== TODO : START ========== ###
# Part 4(b): Implement the random forest classifier and adjust the number of samples used in bootstrap sampling.
print('Classifying using Random Forest...')
max_number_of_samples = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
best_train_error = np.inf
best_test_error = np.inf
best_no_of_samples = None

# Classifying the 'best' classifier based on test_error
for maxsamples in max_number_of_samples:
    random_forest_classifier = RandomForestClassifier(criterion='entropy', random_state=0, max_samples=maxsamples)
    random_forest_classifier.fit(X, y)
    training_error, test_error = error(random_forest_classifier, X, y)
    if test_error < best_test_error:
        best_train_error = training_error
        best_test_error = test_error
        best_no_of_samples = maxsamples

print(f"The best number of samples is {best_no_of_samples} with a training error of {best_train_error} and the testing error is {best_test_error}")
    
### ========== TODO : END ========== ###

Classifying using Random Forest...
The best number of samples is 0.3 with a training error of 0.09427065026362039 and the testing error is 0.1874825174825175


In [106]:
### ========== TODO : START ========== ###
# Part 4(c): Implement the random forest classifier and adjust the number of features for each decision tree.
print('Classifying using Random Forest...')
max_number_of_features = [1,2,3,4,5,6,7]
best_max_features = None
best_train_error = np.inf
best_test_error = np.inf

for maxfeatures in max_number_of_features:
    random_forest_classifier = RandomForestClassifier(criterion='entropy', random_state=0, max_samples=best_no_of_samples, max_features = maxfeatures)
    random_forest_classifier.fit(X,y)
    training_error, testing_error = error(random_forest_classifier, X, y)
    if testing_error < best_test_error:
        best_train_error = training_error
        best_test_error = testing_error
        best_max_features = maxfeatures
    
print(f"The best number of features is {best_max_features} with a training error of {best_train_error} and the testing error is {best_test_error}")
### ========== TODO : END ========== ###

Classifying using Random Forest...
The best number of features is 3 with a training error of 0.09481546572934976 and the testing error is 0.18678321678321677
